In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math

In [3]:
data = pd.read_csv('titanic_csv (1).csv', index_col=0)

In [4]:
type(data)

pandas.core.frame.DataFrame

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  home.dest  745 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 132.9+ KB


In [6]:
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
1,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,"St Louis, MO"
2,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"
5,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"


In [7]:
data = data.interpolate()

In [10]:
X = data[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]
y = data['survived']

In [11]:
X = pd.get_dummies(X)
X.head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
1,1,29.0000,0,0,211.3375,1,0,0,0,1
2,1,0.9167,1,2,151.5500,0,1,0,0,1
3,1,2.0000,1,2,151.5500,1,0,0,0,1
4,1,30.0000,1,2,151.5500,0,1,0,0,1
5,1,25.0000,1,2,151.5500,1,0,0,0,1


In [13]:
# Tạo lại dữ liệu huấn luyện và test sau khi bỏ đi các thuộc tính ít quan trọng hơn
X_now = X[['age', 'fare','sex_female','sex_male', 'pclass']]
y_now = data['survived']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_now, y_now,
                                                   test_size=0.3)


### Grid search

In [15]:
# Dùng grid search
from sklearn.model_selection import GridSearchCV

In [17]:
param_grid = {
    'n_estimators':[50,100,200,300],
    'max_features':['auto','sqrt','log2'] 
}

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
CV_rfc = GridSearchCV(estimator=RandomForestClassifier(random_state=1),
                     param_grid=param_grid,
                     cv=5)

In [21]:
CV_rfc.fit(X_train, y_train)

C:\Program Files\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Program Files\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Program Files\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [50, 100, 200, 300]})

In [22]:
print(CV_rfc.best_params_)

{'max_features': 'auto', 'n_estimators': 100}


In [24]:
y_pred = CV_rfc.predict(X_test)

In [26]:
from sklearn import metrics
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.8142493638676844


In [27]:
print('The Training R^2 score is: ', CV_rfc.score(X_train, y_train))
print('The Testing R^2 score is: ', CV_rfc.score(X_test, y_test))

The Training R^2 score is:  0.9890829694323144
The Testing R^2 score is:  0.8142493638676844


### Random search

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [36]:
param_dist = {
    'n_estimators':[50,100,200,300],
    'max_features':['auto','sqrt','log2'] 
}

In [37]:
forest_random = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=1),
    param_distributions=param_dist,
    cv=5)

In [38]:
forest_random.fit(X_train, y_train)

C:\Program Files\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Program Files\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Program Files\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value f

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
                   param_distributions={'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': [50, 100, 200, 300]})

In [39]:
forest_random_best = forest_random.best_estimator_
print('Best Model Parameter: ', forest_random.best_params_)

Best Model Parameter:  {'n_estimators': 100, 'max_features': 'log2'}


In [40]:
y_pred = forest_random.predict(X_test)

In [42]:
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.8142493638676844


In [43]:
print('The Training R^2 score is: ', forest_random.score(X_train, y_train))
print('The Testing R^2 score is: ', forest_random.score(X_test, y_test))

The Training R^2 score is:  0.9890829694323144
The Testing R^2 score is:  0.8142493638676844


In [44]:
# Model bị overfitting